# 🏛️ Archimedes Chess AI - One-Click Setup

**High-Performance Chess AI with ResNet, AMP, and Advanced MCTS**

## 🚀 Quick Start

1. **Aktiviere GPU**: `Laufzeit` → `Laufzeittyp ändern` → `Hardwarebeschleuniger: GPU`
2. **Führe die Setup-Zelle aus** (unten)
3. **Fertig!** Das Training startet automatisch

## ✨ Features (v2.0)

- **ResNet-basiertes TPN**: 10 Residual Blocks mit Batch Normalization
- **Automatic Mixed Precision (AMP)**: 2-3x schnelleres Training
- **Time-based Iterative Deepening**: Intelligente Suchtiefensteuerung
- **LRU Transposition Table**: Effiziente Cache-Verwaltung
- **Q-Value Normalization**: Stabilere MCTS-Suche
- **Advanced Schedulers**: CosineAnnealingWarmRestarts für bessere Konvergenz

---

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════
# 🎯 ONE-CLICK SETUP & TRAINING
# ═══════════════════════════════════════════════════════════════════════════
# Diese Zelle macht ALLES:
# 1. Installiert Dependencies
# 2. Klont das Repository (oder verwendet hochgeladene Dateien)
# 3. Überprüft GPU-Verfügbarkeit
# 4. Startet das Training mit optimalen Parametern
# ═══════════════════════════════════════════════════════════════════════════

import os
import sys
from pathlib import Path

print("\n" + "="*70)
print("🏛️  ARCHIMEDES CHESS AI - ONE-CLICK SETUP")
print("="*70 + "\n")

# ─────────────────────────────────────────────────────────────────────────
# STEP 1: Install Dependencies
# ─────────────────────────────────────────────────────────────────────────
print("📦 [1/4] Installing dependencies...\n")

# Check if we're in Colab
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    # Install PyTorch with CUDA support
    !pip install -q torch torchvision --index-url https://download.pytorch.org/whl/cu118
    
    # Install other dependencies
    !pip install -q chess python-chess torch-geometric tqdm psutil
    
    print("✅ Dependencies installed!\n")
else:
    print("⚠️  Not running in Colab. Assuming dependencies are already installed.\n")

# ─────────────────────────────────────────────────────────────────────────
# STEP 2: Repository Setup
# ─────────────────────────────────────────────────────────────────────────
print("📂 [2/4] Setting up repository...\n")

# Option A: Clone from GitHub (replace with your repo URL)
REPO_URL = ""  # Leave empty if uploading manually

if REPO_URL:
    print(f"   Cloning from: {REPO_URL}")
    !git clone {REPO_URL} archimedes
    %cd archimedes
    print("✅ Repository cloned!\n")
else:
    # Option B: Check if files are already present
    if Path("train_end_to_end.py").exists():
        print("✅ Repository files found!\n")
    else:
        print("⚠️  Repository not found!")
        print("   Please either:")
        print("   1. Set REPO_URL above and re-run this cell, OR")
        print("   2. Upload the project files to Colab manually\n")
        sys.exit(1)

# ─────────────────────────────────────────────────────────────────────────
# STEP 3: System Check
# ─────────────────────────────────────────────────────────────────────────
print("🔍 [3/4] Checking system...\n")

import torch

print("   System Information:")
print(f"   ├─ Python: {sys.version.split()[0]}")
print(f"   ├─ PyTorch: {torch.__version__}")
print(f"   └─ CUDA Available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"\n   🎮 GPU Information:")
    print(f"   ├─ Name: {torch.cuda.get_device_name(0)}")
    print(f"   ├─ CUDA Version: {torch.version.cuda}")
    props = torch.cuda.get_device_properties(0)
    print(f"   ├─ Memory: {props.total_memory / 1024**3:.1f} GB")
    print(f"   ├─ Compute Capability: {props.major}.{props.minor}")
    print(f"   └─ AMP Support: {'✅ Yes' if props.major >= 7 else '❌ No (requires CC 7.0+)'}")
    
    # Determine optimal batch size based on GPU
    gpu_mem_gb = props.total_memory / 1024**3
    if gpu_mem_gb >= 40:  # A100
        batch_size = 64
        num_workers = 2
    elif gpu_mem_gb >= 15:  # T4, V100
        batch_size = 32
        num_workers = 1
    else:  # Smaller GPUs
        batch_size = 16
        num_workers = 1
    
    use_amp = props.major >= 7  # AMP requires Compute Capability 7.0+
    
    print(f"\n   📊 Recommended Settings:")
    print(f"   ├─ Batch Size: {batch_size}")
    print(f"   ├─ Workers: {num_workers}")
    print(f"   └─ AMP: {'Enabled' if use_amp else 'Disabled'}")
else:
    print("\n   ⚠️  No GPU detected!")
    print("   Please enable GPU: Laufzeit → Laufzeittyp ändern → GPU")
    print("   Training will be VERY slow on CPU!\n")
    batch_size = 8
    num_workers = 1
    use_amp = False

print("\n✅ System check complete!\n")

# ─────────────────────────────────────────────────────────────────────────
# STEP 4: Start Training
# ─────────────────────────────────────────────────────────────────────────
print("🚀 [4/4] Starting training...\n")
print("="*70)

# Training parameters (adjust as needed)
TOTAL_GAMES = 100          # Number of self-play games
TRAINING_ITERATIONS = 500  # Number of training iterations
WARMUP_GAMES = 20          # Warmup games before training
SCHEDULER = "cosine"       # Learning rate scheduler: cosine, plateau, none

# Build command
cmd = f"""python train_end_to_end.py \
    --num-workers {num_workers} \
    --total-games {TOTAL_GAMES} \
    --training-iterations {TRAINING_ITERATIONS} \
    --batch-size {batch_size} \
    --warmup-games {WARMUP_GAMES} \
    --scheduler {SCHEDULER} \
    --checkpoint-dir ./checkpoints \
    --save-every 10 \
    {'--use-amp' if use_amp else '--no-amp'}
"""

print(f"\n📝 Training Configuration:")
print(f"   ├─ Total Games: {TOTAL_GAMES}")
print(f"   ├─ Training Iterations: {TRAINING_ITERATIONS}")
print(f"   ├─ Batch Size: {batch_size}")
print(f"   ├─ Workers: {num_workers}")
print(f"   ├─ Warmup Games: {WARMUP_GAMES}")
print(f"   ├─ Scheduler: {SCHEDULER}")
print(f"   └─ AMP: {'Enabled' if use_amp else 'Disabled'}")
print("\n" + "="*70 + "\n")

# Execute training
!{cmd}

print("\n" + "="*70)
print("✅ TRAINING COMPLETE!")
print("="*70)
print("\n📁 Checkpoints saved in: ./checkpoints/")
print("📊 Training logs saved in: ./checkpoints/training_logs.db")
print("\n💡 Next Steps:")
print("   1. Download checkpoints: Files → checkpoints → Download")
print("   2. View logs with dashboard.py (requires local setup)")
print("   3. Evaluate model with evaluate_elo.py")
print("\n🎉 Happy training!\n")

---

## 📊 Optional: View Training Progress

If you want to monitor training in real-time, you can use the following cell to display basic metrics:

In [ ]:
# Optional: Display training metrics from database
import sqlite3
import pandas as pd
from pathlib import Path

db_path = Path("./checkpoints/training_logs.db")

if db_path.exists():
    conn = sqlite3.connect(str(db_path))
    
    # Get epoch summary
    print("📈 Training Progress:\n")
    df = pd.read_sql_query(
        "SELECT epoch, total_loss, policy_loss, value_loss, lr, top1_accuracy, top5_accuracy FROM epoch_summary ORDER BY epoch DESC LIMIT 10",
        conn
    )
    print(df.to_string(index=False))
    
    # Get hardware stats
    print("\n\n🖥️  Hardware Utilization (Latest):\n")
    df_hw = pd.read_sql_query(
        "SELECT gpu_utilization_pct, vram_mb, gpu_temp_c, cpu_load_pct, ram_mb FROM hardware_snapshots ORDER BY timestamp DESC LIMIT 1",
        conn
    )
    if not df_hw.empty:
        print(f"   GPU Utilization: {df_hw['gpu_utilization_pct'].iloc[0]:.1f}%")
        print(f"   VRAM Usage: {df_hw['vram_mb'].iloc[0]:.0f} MB")
        print(f"   GPU Temperature: {df_hw['gpu_temp_c'].iloc[0]:.1f}°C")
        print(f"   CPU Load: {df_hw['cpu_load_pct'].iloc[0]:.1f}%")
        print(f"   RAM Usage: {df_hw['ram_mb'].iloc[0]:.0f} MB")
    
    conn.close()
else:
    print("⚠️  No training logs found. Start training first!")

---

## 🎮 Optional: Test the Trained Model

After training, you can test the model by playing a few moves:

In [ ]:
# Optional: Test the trained model
import torch
import chess
from src.archimedes.model import TPN, SAN, PlanToMoveMapper
from src.archimedes.search import ConceptualGraphSearch
from pathlib import Path

# Load checkpoint
checkpoint_path = Path("./checkpoints/latest_checkpoint.pt")

if checkpoint_path.exists():
    print("🎮 Loading trained model...\n")
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Initialize models
    tpn = TPN().to(device)
    san = SAN().to(device)
    mapper = PlanToMoveMapper().to(device)
    
    # Load weights
    checkpoint = torch.load(checkpoint_path, map_location=device)
    tpn.load_state_dict(checkpoint["tpn_state_dict"])
    san.load_state_dict(checkpoint["san_state_dict"])
    mapper.load_state_dict(checkpoint["mapper_state_dict"])
    
    tpn.eval()
    san.eval()
    mapper.eval()
    
    # Create search engine
    search = ConceptualGraphSearch(
        tpn, san, mapper,
        num_simulations=50,
        use_transposition_table=True,
        use_q_normalization=True
    )
    
    # Play a few moves
    board = chess.Board()
    print("Starting position:")
    print(board)
    print("\n" + "="*50 + "\n")
    
    for move_num in range(5):
        result = search.search(board, temperature=0.0)
        move = result["best_move"]
        
        print(f"Move {move_num + 1}: {move}")
        print(f"   Tactical Value: {result['v_tactical'].item():.3f}")
        print(f"   MCTS Stats: NPS={result['mcts_stats'].get('nps', 0):.0f}, Depth={result['mcts_stats'].get('avg_depth', 0):.1f}")
        
        board.push(move)
        print(f"\n{board}\n")
        print("─" * 50 + "\n")
    
    print("✅ Model test complete!")
else:
    print("⚠️  No checkpoint found. Train the model first!")

---

## 📚 Additional Resources

- **GitHub Repository**: [Link to your repo]
- **Documentation**: See README.md for detailed information
- **Paper**: [Link to paper if available]

## 🐛 Troubleshooting

### Out of Memory Error
- Reduce `batch_size` in the training cell
- Disable AMP by setting `use_amp = False`
- Reduce number of residual blocks in model.py

### Training is slow
- Make sure GPU is enabled (Runtime → Change runtime type → GPU)
- Enable AMP if your GPU supports it (Compute Capability 7.0+)
- Reduce `num_simulations` in search engine

### GPU not detected
- Go to: Runtime → Change runtime type → Hardware accelerator → GPU
- Restart runtime and re-run the setup cell

---

**Archimedes Chess AI** - High-Performance Chess Engine with Hybrid Architecture

*Built with ❤️ using PyTorch, ResNet, and Advanced MCTS*